# Part 2: Agentic RAG
## Building Intelligent Agents That Plan, Act, and Cite

In Part 1, we saw that basic RAG fails when action is required. Now we'll fix that!

**What we'll build:**
1. Tool definitions (order API, refund calculator, returns processor)
2. Agent state management with LangGraph
3. ReAct planning loop (Thought → Action → Observation)
4. Citation tracking for every fact
5. Multi-step task solving

**Architecture:**
```
Query → Planner → [Check Order | Calculate Refund | Retrieve Policy] → Synthesize → Cited Response
```

## Setup & Imports

In [1]:
# Install required packages (run once)
# !pip install openai langgraph langchain langchain-core pydantic python-dotenv

In [2]:
import os
import json
from typing import List, Dict, Any, Literal
from datetime import datetime, timedelta
from pydantic import BaseModel, Field
import httpx
from openai import OpenAI
from langgraph.graph import StateGraph, END
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# ⚠️ SSL BYPASS FOR CORPORATE NETWORKS (DEMO ONLY - NOT FOR PRODUCTION)
# This disables SSL certificate verification to work with corporate proxies
# For production, configure proper SSL certificates instead
http_client = httpx.Client(verify=False)

# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=http_client  # SSL bypass for corporate networks
)

print("✅ Setup complete!")
print("⚠️  SSL verification disabled (for demo purposes only)")

✅ Setup complete!
⚠️  SSL verification disabled (for demo purposes only)


## 1. Mock Order Database

In production, this would be a real database. For this demo, we'll use a simple dict.

In [3]:
ORDERS_DB = {
    "12345": {
        "order_id": "12345",
        "customer_email": "john@example.com",
        "items": [
            {"name": "Blue Cotton T-Shirt", "price": 29.99, "quantity": 2},
            {"name": "Denim Jeans", "price": 59.99, "quantity": 1}
        ],
        "total": 119.97,
        "order_date": "2025-01-10",
        "ship_date": "2025-01-11",
        "delivery_date": "2025-01-15",
        "status": "delivered",
        "tracking_number": "1Z999AA10123456784"
    },
    "67890": {
        "order_id": "67890",
        "customer_email": "jane@example.com",
        "items": [
            {"name": "Red Hoodie", "price": 45.00, "quantity": 1}
        ],
        "total": 45.00,
        "order_date": "2025-01-20",
        "ship_date": "2025-01-21",
        "delivery_date": None,
        "status": "in_transit",
        "tracking_number": "1Z999AA10987654321"
    }
}

# Display orders nicely
print("📦 Mock Order Database:\n")
for order_id, order in ORDERS_DB.items():
    print(f"Order #{order_id}:")
    print(f"  Status: {order['status']}")
    print(f"  Total: ${order['total']}")
    print(f"  Items: {', '.join([item['name'] for item in order['items']])}")
    print()

📦 Mock Order Database:

Order #12345:
  Status: delivered
  Total: $119.97
  Items: Blue Cotton T-Shirt, Denim Jeans

Order #67890:
  Status: in_transit
  Total: $45.0
  Items: Red Hoodie



## 2. Tool Definitions

Tools are functions the LLM can call to take actions or get real-time information.

**Key pattern:** Every tool returns a `source` field for citation tracking!

### Tool 1: Check Order Status

In [4]:
def check_order_status(order_id: str) -> Dict[str, Any]:
    """
    Check the status of an order by order ID.
    
    Args:
        order_id: The order ID to check
    
    Returns:
        Order details including status, tracking, and items
    """
    if order_id not in ORDERS_DB:
        return {
            "error": f"Order {order_id} not found",
            "source": "order_api"
        }
    
    order = ORDERS_DB[order_id]
    return {
        "data": order,
        "source": "order_api",
        "retrieved_at": datetime.now().isoformat()
    }

# Test it
test_result = check_order_status("12345")
print("Test: check_order_status('12345')")
print(json.dumps(test_result, indent=2))

Test: check_order_status('12345')
{
  "data": {
    "order_id": "12345",
    "customer_email": "john@example.com",
    "items": [
      {
        "name": "Blue Cotton T-Shirt",
        "price": 29.99,
        "quantity": 2
      },
      {
        "name": "Denim Jeans",
        "price": 59.99,
        "quantity": 1
      }
    ],
    "total": 119.97,
    "order_date": "2025-01-10",
    "ship_date": "2025-01-11",
    "delivery_date": "2025-01-15",
    "status": "delivered",
    "tracking_number": "1Z999AA10123456784"
  },
  "source": "order_api",
  "retrieved_at": "2025-11-05T12:31:08.724922"
}


### Tool 2: Calculate Refund

In [5]:
def calculate_refund(order_id: str, reason: str) -> Dict[str, Any]:
    """
    Calculate refund amount based on order and return reason.
    
    Args:
        order_id: The order ID
        reason: Reason for return (damaged, defective, change_of_mind)
    
    Returns:
        Refund calculation details
    """
    if order_id not in ORDERS_DB:
        return {"error": f"Order {order_id} not found", "source": "refund_calculator"}
    
    order = ORDERS_DB[order_id]
    total = order["total"]
    
    # Apply business rules
    if reason.lower() in ["damaged", "defective"]:
        refund_amount = total
        restocking_fee = 0.0
    else:  # change_of_mind
        restocking_fee = total * 0.15
        refund_amount = total - restocking_fee
    
    return {
        "data": {
            "order_id": order_id,
            "original_amount": total,
            "restocking_fee": restocking_fee,
            "refund_amount": refund_amount,
            "reason": reason
        },
        "source": "refund_calculator",
        "rule": "Full refund for damaged/defective; 15% fee for change of mind"
    }

# Test it
test_result = calculate_refund("12345", "damaged")
print("Test: calculate_refund('12345', 'damaged')")
print(json.dumps(test_result, indent=2))

Test: calculate_refund('12345', 'damaged')
{
  "data": {
    "order_id": "12345",
    "original_amount": 119.97,
    "restocking_fee": 0.0,
    "refund_amount": 119.97,
    "reason": "damaged"
  },
  "source": "refund_calculator",
  "rule": "Full refund for damaged/defective; 15% fee for change of mind"
}


### Tool 3: Process Return

In [6]:
def process_return(order_id: str, reason: str, items: str = "all") -> Dict[str, Any]:
    """
    Process a return request for an order.
    
    Args:
        order_id: The order ID
        reason: Reason for return
        items: Which items to return (default: all)
    
    Returns:
        Return request confirmation
    """
    if order_id not in ORDERS_DB:
        return {"error": f"Order {order_id} not found", "source": "returns_api"}
    
    order = ORDERS_DB[order_id]
    
    # Check if order is delivered and within 30 days
    if order["delivery_date"]:
        delivery_date = datetime.fromisoformat(order["delivery_date"])
        days_since_delivery = (datetime.now() - delivery_date).days
        
        if days_since_delivery > 30:
            return {
                "error": "Return window expired (30 days)",
                "source": "returns_api",
                "policy_reference": "return_policy.md#eligibility"
            }
    
    # Calculate refund
    refund_calc = calculate_refund(order_id, reason)
    if "error" in refund_calc:
        return refund_calc
    
    # Generate return ID
    return_id = f"RET-{order_id}-{datetime.now().strftime('%Y%m%d')}"
    estimated_refund = (datetime.now() + timedelta(days=10)).strftime('%Y-%m-%d')
    
    return {
        "data": {
            "return_id": return_id,
            "order_id": order_id,
            "status": "approved",
            "refund_amount": refund_calc["data"]["refund_amount"],
            "estimated_refund_date": estimated_refund,
            "next_steps": "Return label sent to email. Ship within 5 business days."
        },
        "source": "returns_api",
        "processed_at": datetime.now().isoformat()
    }

# Test it
test_result = process_return("12345", "damaged")
print("Test: process_return('12345', 'damaged')")
print(json.dumps(test_result, indent=2))

Test: process_return('12345', 'damaged')
{
  "error": "Return window expired (30 days)",
  "source": "returns_api",
  "policy_reference": "return_policy.md#eligibility"
}


### Tool 4: Retrieve Policy (Reuse RAG from Part 1)

In [7]:
# Simple policy retrieval (in production, use the full RAG from Part 1)
KNOWLEDGE_BASE = {
    "return_policy": "Items can be returned within 30 days of delivery. Damaged/defective items: Full refund. Change of mind: Refund minus 15% restocking fee.",
    "shipping_policy": "Standard shipping: 5-7 business days. Express: 2-3 days. Free shipping on orders over $50."
}

def retrieve_policy(query: str) -> Dict[str, Any]:
    """
    Retrieve relevant policy information from knowledge base.
    
    Args:
        query: The policy question
    
    Returns:
        Relevant policy excerpts
    """
    # Simple keyword matching for demo (use full RAG in production)
    results = []
    if "return" in query.lower():
        results.append({"policy": "return_policy", "text": KNOWLEDGE_BASE["return_policy"]})
    if "ship" in query.lower():
        results.append({"policy": "shipping_policy", "text": KNOWLEDGE_BASE["shipping_policy"]})
    
    return {
        "data": {"query": query, "excerpts": results},
        "source": "knowledge_base",
        "retrieved_at": datetime.now().isoformat()
    }

# Test it
test_result = retrieve_policy("What is the return policy?")
print("Test: retrieve_policy('What is the return policy?')")
print(json.dumps(test_result, indent=2))

Test: retrieve_policy('What is the return policy?')
{
  "data": {
    "query": "What is the return policy?",
    "excerpts": [
      {
        "policy": "return_policy",
        "text": "Items can be returned within 30 days of delivery. Damaged/defective items: Full refund. Change of mind: Refund minus 15% restocking fee."
      }
    ]
  },
  "source": "knowledge_base",
  "retrieved_at": "2025-11-05T12:31:12.661002"
}


## 3. Tool Registry for OpenAI

Define tools in OpenAI's function calling format so the LLM knows when and how to use them.

In [8]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "check_order_status",
            "description": "Check the status of an order by order ID. Use this when customer asks about order status, tracking, or delivery.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {"type": "string", "description": "The order ID to check"}
                },
                "required": ["order_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_refund",
            "description": "Calculate the refund amount for a return based on reason. Use this to determine how much customer will receive back.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {"type": "string", "description": "The order ID"},
                    "reason": {
                        "type": "string",
                        "enum": ["damaged", "defective", "change_of_mind"],
                        "description": "Reason for return"
                    }
                },
                "required": ["order_id", "reason"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "process_return",
            "description": "Process a return request. Only call this after verifying eligibility and calculating refund.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {"type": "string", "description": "The order ID"},
                    "reason": {"type": "string", "description": "Reason for return"},
                    "items": {"type": "string", "description": "Items to return", "default": "all"}
                },
                "required": ["order_id", "reason"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_policy",
            "description": "Retrieve policy information from knowledge base. Use when customer asks about policies, procedures, or general questions.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The policy question"}
                },
                "required": ["query"]
            }
        }
    }
]

# Tool execution map
TOOL_MAP = {
    "check_order_status": check_order_status,
    "calculate_refund": calculate_refund,
    "process_return": process_return,
    "retrieve_policy": retrieve_policy
}

print(f"✅ Registered {len(TOOLS)} tools:")
for tool in TOOLS:
    print(f"  - {tool['function']['name']}")

✅ Registered 4 tools:
  - check_order_status
  - calculate_refund
  - process_return
  - retrieve_policy


## 4. Agent State

Define the state that persists throughout the agent's execution.

In [9]:
class AgentState(BaseModel):
    """State for the agent conversation."""
    query: str
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    tool_results: List[Dict[str, Any]] = Field(default_factory=list)
    sources: List[str] = Field(default_factory=list)
    steps: int = 0
    max_steps: int = 10
    final_response: str | None = None

print("✅ AgentState defined with fields:")
for field_name in AgentState.model_fields.keys():
    print(f"  - {field_name}")

✅ AgentState defined with fields:
  - query
  - messages
  - tool_results
  - sources
  - steps
  - max_steps
  - final_response


## 5. Agent Nodes

Define the three key nodes in our agent graph:
1. **Planner** - Decides what to do next
2. **Tool Executor** - Runs the tools
3. **Responder** - Generates final answer with citations

### Node 1: Planner (LLM decides next action)

In [10]:
def call_model(state: AgentState) -> AgentState:
    """Call LLM to plan next action."""
    print(f"\n🧠 Planning (Step {state.steps + 1}/{state.max_steps})...")
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are an intelligent customer support agent with access to tools.

Your capabilities:
- Check order status and tracking
- Calculate refunds based on return policy
- Process return requests
- Retrieve policy information

Always:
1. Use tools to get accurate, real-time information
2. Cite sources for every fact (e.g., [Source: order_api])
3. Be helpful and professional
4. If you don't have enough information, ask clarifying questions

When processing returns:
1. First check order status
2. Then calculate refund
3. Finally process return if customer confirms

Never make up order details or policy information."""
            }
        ] + state.messages,
        tools=TOOLS,
        tool_choice="auto"
    )
    
    message = response.choices[0].message
    state.messages.append({
        "role": "assistant",
        "content": message.content,
        "tool_calls": [
            {
                "id": tc.id,
                "type": tc.type,
                "function": {
                    "name": tc.function.name,
                    "arguments": tc.function.arguments
                }
            }
            for tc in (message.tool_calls or [])
        ] if message.tool_calls else None
    })
    state.steps += 1
    
    return state

### Node 2: Tool Executor (Run the tools)

In [11]:
def execute_tools(state: AgentState) -> AgentState:
    """Execute tool calls from LLM."""
    last_message = state.messages[-1]
    tool_calls = last_message.get("tool_calls", [])
    
    if not tool_calls:
        return state
    
    print(f"🔧 Executing {len(tool_calls)} tool(s)...")
    
    for tool_call in tool_calls:
        function_name = tool_call["function"]["name"]
        function_args = json.loads(tool_call["function"]["arguments"])
        
        print(f"   → {function_name}({', '.join(f'{k}={v}' for k, v in function_args.items())})") 
        
        # Execute tool
        tool_func = TOOL_MAP[function_name]
        result = tool_func(**function_args)
        
        # Track source
        if "source" in result:
            state.sources.append(result["source"])
        
        # Add tool result to messages
        state.messages.append({
            "role": "tool",
            "tool_call_id": tool_call["id"],
            "name": function_name,
            "content": json.dumps(result)
        })
        
        state.tool_results.append({
            "tool": function_name,
            "args": function_args,
            "result": result
        })
    
    return state

### Node 3: Responder (Generate final answer with citations)

In [12]:
def generate_response(state: AgentState) -> AgentState:
    """Generate final response with citations."""
    print("\n✍️  Generating final response with citations...")
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=state.messages + [
            {
                "role": "system",
                "content": """Generate a final response to the customer.

CRITICAL: Every factual claim MUST include a citation in square brackets.
Examples:
- "Your order shipped on Jan 15 [Source: order_api]"
- "Full refund for damaged items [Source: return_policy.md]"
- "Refund amount: $89.99 [Source: refund_calculator]"

If you cannot cite a source for a claim, do not make the claim.
Be helpful, professional, and complete."""
            }
        ],
        temperature=0.3
    )
    
    state.final_response = response.choices[0].message.content
    return state

### Control Flow (Decide when to use tools vs respond)

In [13]:
def should_continue(state: AgentState) -> Literal["tools", "respond"]:
    """Decide whether to use tools or generate final response."""
    last_message = state.messages[-1]
    
    # Check if we hit max steps
    if state.steps >= state.max_steps:
        return "respond"
    
    # Check if LLM wants to use tools
    if last_message.get("tool_calls"):
        return "tools"
    
    return "respond"

## 6. Build the Agent Graph

Assemble everything into a LangGraph state machine.

In [14]:
def build_agent():
    """Build the LangGraph agent."""
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("planner", call_model)
    workflow.add_node("tools", execute_tools)
    workflow.add_node("responder", generate_response)
    
    # Add edges
    workflow.set_entry_point("planner")
    workflow.add_conditional_edges(
        "planner",
        should_continue,
        {
            "tools": "tools",
            "respond": "responder"
        }
    )
    workflow.add_edge("tools", "planner")  # Loop back to planner after tools
    workflow.add_edge("responder", END)
    
    return workflow.compile()

print("✅ Agent graph built!")
print("\nFlow: planner → tools → planner → tools → ... → responder → END")

✅ Agent graph built!

Flow: planner → tools → planner → tools → ... → responder → END


## 7. Helper Function to Run Queries

In [15]:
def run_agent_query(query: str):
    """Run a query through the agentic RAG system."""
    print(f"\n{'='*80}")
    print(f"❓ Query: {query}")
    print("="*80)
    
    # Build agent
    agent = build_agent()
    
    # Initialize state
    initial_state = AgentState(
        query=query,
        messages=[{"role": "user", "content": query}]
    )
    
    # Run agent - returns a dict, not AgentState object
    result = agent.invoke(initial_state)
    
    # Display results
    print(f"\n💬 Final Response:")
    print("-"*80)
    display(Markdown(result["final_response"]))
    print("-"*80)
    
    print(f"\n📊 Agent Statistics:")
    print(f"   Steps taken: {result['steps']}")
    print(f"   Tools used: {len(result['tool_results'])}")
    print(f"   Sources cited: {list(set(result['sources']))}")
    
    print(f"\n🔍 Tool Execution Trace:")
    for i, tool_result in enumerate(result['tool_results'], 1):
        print(f"   {i}. {tool_result['tool']}({tool_result['args']})")
        print(f"      → Source: {tool_result['result'].get('source', 'N/A')}")
    
    return result

## 8. Demo: Simple Order Status ✅

Let's start with a simple query that requires one tool call.

In [16]:
result1 = run_agent_query("Where is my order #12345?")


❓ Query: Where is my order #12345?

🧠 Planning (Step 1/10)...
🔧 Executing 1 tool(s)...
   → check_order_status(order_id=12345)

🧠 Planning (Step 2/10)...

✍️  Generating final response with citations...

💬 Final Response:
--------------------------------------------------------------------------------


Your order #12345 has been delivered [Source: order_api]. Here are the details:

- **Items Ordered:**
  - Blue Cotton T-Shirt (Quantity: 2)
  - Denim Jeans (Quantity: 1)
- **Total Amount:** $119.97 [Source: order_api]
- **Order Date:** January 10, 2025 [Source: order_api]
- **Ship Date:** January 11, 2025 [Source: order_api]
- **Delivery Date:** January 15, 2025 [Source: order_api]
- **Status:** Delivered [Source: order_api]
- **Tracking Number:** 1Z999AA10123456784 [Source: order_api]

If you have any further questions or need assistance, please let us know!

--------------------------------------------------------------------------------

📊 Agent Statistics:
   Steps taken: 2
   Tools used: 1
   Sources cited: ['order_api']

🔍 Tool Execution Trace:
   1. check_order_status({'order_id': '12345'})
      → Source: order_api


### 🎯 What Just Happened?

Notice how the agent:
1. ✅ **Planned**: Recognized it needed to check order status
2. ✅ **Used Tool**: Called `check_order_status(12345)`
3. ✅ **Got Real Data**: Retrieved actual order information from "API"
4. ✅ **Cited Source**: Every fact includes `[Source: order_api]`
5. ✅ **Synthesized**: Generated helpful, accurate response

**This is already better than baseline RAG!**

## 9. Demo: The Query That Broke Baseline RAG ✅

Remember this query from Part 1 that **failed** with baseline RAG?

**Baseline RAG could only:**
- ❌ Retrieve return policy text
- ❌ NOT check if order exists
- ❌ NOT verify it's eligible for return
- ❌ NOT actually process the return

**Let's see how Agentic RAG handles it:**

In [17]:
result2 = run_agent_query("I want to return order #12345 because it arrived damaged. Can you process this?")


❓ Query: I want to return order #12345 because it arrived damaged. Can you process this?

🧠 Planning (Step 1/10)...
🔧 Executing 1 tool(s)...
   → check_order_status(order_id=12345)

🧠 Planning (Step 2/10)...
🔧 Executing 1 tool(s)...
   → calculate_refund(order_id=12345, reason=damaged)

🧠 Planning (Step 3/10)...

✍️  Generating final response with citations...

💬 Final Response:
--------------------------------------------------------------------------------


Your order #12345 was delivered on January 15, 2025 [Source: order_api]. Since you are returning the order because it arrived damaged, you are eligible for a full refund of $119.97 [Source: refund_calculator]. Please let me know if you would like me to proceed with processing this return.

--------------------------------------------------------------------------------

📊 Agent Statistics:
   Steps taken: 3
   Tools used: 2
   Sources cited: ['order_api', 'refund_calculator']

🔍 Tool Execution Trace:
   1. check_order_status({'order_id': '12345'})
      → Source: order_api
   2. calculate_refund({'order_id': '12345', 'reason': 'damaged'})
      → Source: refund_calculator


### 🎯 Multi-Step Planning!

Look at what the agent did:
1. ✅ **Step 1**: Called `check_order_status(12345)` - verify order exists
2. ✅ **Step 2**: Called `calculate_refund(12345, "damaged")` - get refund amount
3. ✅ **Step 3**: Synthesized answer with **multiple citations**

**Compare to baseline RAG:**
- ❌ Baseline: Just retrieved policy text, no action
- ✅ Agentic: Checked order + calculated refund + cited sources

**This is the power of Agentic RAG!**

## 10. Demo: Policy Question (Uses RAG Tool) ✅

In [19]:
result3 = run_agent_query("What is your return policy for defective items?")


❓ Query: What is your return policy for defective items?

🧠 Planning (Step 1/10)...
🔧 Executing 1 tool(s)...
   → retrieve_policy(query=return policy for defective items)

🧠 Planning (Step 2/10)...

✍️  Generating final response with citations...

💬 Final Response:
--------------------------------------------------------------------------------


Our return policy for defective items allows you to return the item within 30 days of delivery for a full refund [Source: knowledge_base]. If you have any other questions or need further assistance with your return, please don't hesitate to reach out!

--------------------------------------------------------------------------------

📊 Agent Statistics:
   Steps taken: 2
   Tools used: 1
   Sources cited: ['knowledge_base']

🔍 Tool Execution Trace:
   1. retrieve_policy({'query': 'return policy for defective items'})
      → Source: knowledge_base


### 🎯 Smart Tool Selection

Notice:
- ✅ Agent recognized this is a **policy question** (not order-specific)
- ✅ Used `retrieve_policy` tool (our RAG component!)
- ✅ Didn't unnecessarily call order API
- ✅ Still cited the source

**The agent is choosing the right tool for each task!**

## 11. Demo: Edge Case - Non-Existent Order ✅

In [20]:
result4 = run_agent_query("What's the status of order #99999?")


❓ Query: What's the status of order #99999?

🧠 Planning (Step 1/10)...
🔧 Executing 1 tool(s)...
   → check_order_status(order_id=99999)

🧠 Planning (Step 2/10)...

✍️  Generating final response with citations...

💬 Final Response:
--------------------------------------------------------------------------------


I'm sorry, but it seems that order #99999 cannot be found in our system [Source: order_api]. Could you please double-check the order number or provide any additional details that might help us locate your order? If you have any further questions or need assistance, feel free to reach out.

--------------------------------------------------------------------------------

📊 Agent Statistics:
   Steps taken: 2
   Tools used: 1
   Sources cited: ['order_api']

🔍 Tool Execution Trace:
   1. check_order_status({'order_id': '99999'})
      → Source: order_api


### 🎯 Graceful Error Handling

Important:
- ✅ Agent tried to check order status
- ✅ Received error: "Order not found"
- ✅ **Did NOT hallucinate** fake order details
- ✅ Provided helpful guidance to customer

**This is critical for production systems!**

## 12. Demo: Complex Multi-Request ✅

In [21]:
result5 = run_agent_query(
    "I received order #12345 but the shirt is damaged. I want a refund. "
    "Also, when will order #67890 arrive?"
)


❓ Query: I received order #12345 but the shirt is damaged. I want a refund. Also, when will order #67890 arrive?

🧠 Planning (Step 1/10)...
🔧 Executing 3 tool(s)...
   → check_order_status(order_id=12345)
   → check_order_status(order_id=67890)
   → calculate_refund(order_id=12345, reason=damaged)

🧠 Planning (Step 2/10)...

✍️  Generating final response with citations...

💬 Final Response:
--------------------------------------------------------------------------------


Here's the information regarding your orders:

1. **Order #12345**:
   - This order was delivered on January 15, 2025 [Source: order_api].
   - You are eligible for a full refund of $119.97 due to the damaged shirt, with no restocking fees applied [Source: refund_calculator].

2. **Order #67890**:
   - This order is currently in transit. It was shipped on January 21, 2025, and the tracking number is 1Z999AA10987654321 [Source: order_api].

Please let us know if you would like to proceed with the return for Order #12345, and we can assist you further with the refund process. If you have any other questions or need additional assistance, feel free to reach out.

--------------------------------------------------------------------------------

📊 Agent Statistics:
   Steps taken: 2
   Tools used: 3
   Sources cited: ['order_api', 'refund_calculator']

🔍 Tool Execution Trace:
   1. check_order_status({'order_id': '12345'})
      → Source: order_api
   2. check_order_status({'order_id': '67890'})
      → Source: order_api
   3. calculate_refund({'order_id': '12345', 'reason': 'damaged'})
      → Source: refund_calculator


### 🎯 Multi-Request Handling

Impressive:
- ✅ Recognized **two separate requests** in one query
- ✅ Called multiple tools (3-4 tool calls)
- ✅ Organized response clearly (addressed both issues)
- ✅ Cited all sources for all claims

**This is production-grade capability!**

## 🎉 Summary: Baseline RAG vs Agentic RAG

| Capability | Baseline RAG | Agentic RAG |
|------------|--------------|-------------|
| **Answer policy questions** | ✅ Yes | ✅ Yes |
| **Check real-time order status** | ❌ No | ✅ Yes |
| **Calculate refunds** | ❌ No | ✅ Yes |
| **Process returns** | ❌ No | ✅ Yes |
| **Multi-step planning** | ❌ No | ✅ Yes |
| **Source citations** | Partial | ✅ Always |
| **Verify data before responding** | ❌ No | ✅ Yes |
| **Handle complex queries** | ❌ Limited | ✅ Yes |

---

## Key Concepts Learned

### 1. ReAct Pattern
```
Thought: "I need to check if order exists"
Action: call check_order_status(12345)
Observation: Order found, delivered on 2025-01-15
Thought: "Now I can calculate the refund"
Action: call calculate_refund(12345, "damaged")
Observation: Refund = $119.97
Thought: "I have enough info to respond"
```

### 2. Tool Design
Every tool returns:
```python
{
  "data": {...},           # Actual result
  "source": "order_api",   # For citations!
  "retrieved_at": "..."    # Audit trail
}
```

### 3. Citations
Every fact must have a source:
- ✅ "Order shipped Jan 15 [Source: order_api]"
- ✅ "Full refund $119.97 [Source: refund_calculator]"
- ❌ "Order shipped recently" (no source = refused)

### 4. State Management
LangGraph tracks:
- Conversation history
- Tool results
- Sources for citations
- Steps taken (with max limit)

---

## When to Use Agentic RAG?

**Use Agentic RAG when:**
- ✅ Multi-step reasoning required
- ✅ Need to take actions (not just retrieve info)
- ✅ Verification/validation needed
- ✅ Source attribution critical
- ✅ Complex decision-making

**Stick with basic RAG when:**
- ✅ Simple Q&A
- ✅ No external tools needed
- ✅ Ultra-low latency required (<1s)
- ✅ Very cost-sensitive at massive scale

---

## Next Steps

**Try adding:**
1. New tool: `send_email(to, subject, body)`
2. New tool: `check_inventory(product_id)`
3. Evaluation with RAGAS (faithfulness, precision)
4. Deploy as FastAPI service
5. Add caching for cost optimization

**Resources:**
- LangGraph: https://langchain-ai.github.io/langgraph/
- OpenAI Function Calling: https://platform.openai.com/docs/guides/function-calling
- RAGAS: https://github.com/explodinggradients/ragas

---

**🎉 Congratulations! You've built a production-capable Agentic RAG system!**